# Ugesco : enrich and desambiguate the Samnang-Hans' JSON

In [2]:
from ugesco import *
import warnings
warnings.filterwarnings('ignore') #évite les alertes lors de l'import d'Ugesco.py


In [3]:
file = "data/jsondata_ugesco.json"
data = json_to_df(file)

In [4]:
# exclude image classifications and temporal values for the moment
data = data[data.columns.drop(list(data.filter(regex='imageclassification|temporal')))]

In [5]:
# reshape the dataframe verticaly
data = data.set_index('beeldid')
data.columns = data.columns.str.split('_', expand=True)
data = data.stack().reset_index(level=1, drop=True).reset_index()
data.columns = ['beeldid', 'spatial_value', 'spatial_key', 'to_drop']
data.drop('to_drop', 1, inplace=True)

In [6]:
#join/merge with phototheque_pallas to get the locations. 
#The csv is gzipped to get around the 100MB file weigth limitation of Github
phototeque = pd.read_csv("data/phototheque_pallas.csv.gz", compression="gzip", encoding="utf8", dtype={'beeldid': str})

data = pd.merge(data, phototeque, how='left', on=['beeldid'], suffixes=['', '_x'])

In [7]:
# Pictures that contains more than a loc in the thesaurus descriptors

data[data['loc_qid'].str.contains(",", na = False)]

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name
122,00028254,la prison St . Gilles,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Entrée de la prison St. Gilles.- [Pangrapho] ...,Camps et prisons : [1940-1945]. - 14 photos,1626.0,"saint-gilles (bruxelles, belgique)--prison",Q31,Belgium,"Q237674,Q202954","Saint-Gilles - Sint-Gillis,Verviers"
123,00028265,Camp de Beverloo,LOC_MISC,http://pallas.cegesoma.be/pls/opac/opac.search...,"Vue prise de haut d'un ""Mirador"" du camp de P....",Camps et prisons : [1940-1945]. - 14 photos,1626.0,"saint-gilles (bruxelles, belgique)--prison",Q31,Belgium,"Q237674,Q202954","Saint-Gilles - Sint-Gillis,Verviers"
339,00040210,gare du Midi,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,"La Micheline, automobile pour voie ferrée, rou...",Chemins de fer : [...-1939]. - 41 photos,1668.0,chemins de fer--belgique,Q31,Belgium,"Q239,Q573179,Q12892,Q12996","Ostend,Brussels,Couillet,Antwerp"
340,00040210,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"La Micheline, automobile pour voie ferrée, rou...",Chemins de fer : [...-1939]. - 41 photos,1668.0,chemins de fer--belgique,Q31,Belgium,"Q239,Q573179,Q12892,Q12996","Ostend,Brussels,Couillet,Antwerp"
341,00040211,la Gare du Midi,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,"La voiture automobile sur pneus, qui sera exam...",Chemins de fer : [...-1939]. - 41 photos,1668.0,chemins de fer--belgique,Q31,Belgium,"Q239,Q573179,Q12892,Q12996","Ostend,Brussels,Couillet,Antwerp"
342,00040211,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"La voiture automobile sur pneus, qui sera exam...",Chemins de fer : [...-1939]. - 41 photos,1668.0,chemins de fer--belgique,Q31,Belgium,"Q239,Q573179,Q12892,Q12996","Ostend,Brussels,Couillet,Antwerp"
343,00040214,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. La Société nationale des Chemins de...,Chemins de fer : [...-1939]. - 41 photos,1668.0,chemins de fer--belgique,Q31,Belgium,"Q239,Q573179,Q12892,Q12996","Brussels,Ostend,Couillet,Antwerp"
344,00040218,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Grâce et puissance. La charmante petite Reine ...,Chemins de fer : [...-1939]. - 41 photos,1668.0,chemins de fer--belgique,Q31,Belgium,"Q239,Q573179,Q12892,Q12996","Ostend,Brussels,Couillet,Antwerp"
345,00040218,Dunkerque,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Grâce et puissance. La charmante petite Reine ...,Chemins de fer : [...-1939]. - 41 photos,1668.0,chemins de fer--belgique,Q31,Belgium,"Q239,Q573179,Q12892,Q12996","Ostend,Brussels,Couillet,Antwerp"
346,00040233,Ostende,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Vacances de Pâques. C'est l'époque de la grand...,Chemins de fer : [...-1939]. - 41 photos,1668.0,chemins de fer--belgique,Q31,Belgium,"Q239,Q573179,Q12892,Q12996","Ostend,Brussels,Couillet,Antwerp"


In [ ]:
#test Rosette API. Max 10 000 calls a month and 1000 a day
        
#data['rosette'] = data['LEGEND'].apply(rosette)

In [ ]:
# Some descriptive stats on columns
#data.describe(include="all")

In [ ]:
# More data profiling
#import pandas_profiling
#pandas_profiling.ProfileReport(data)

In [ ]:
#data.to_csv("C:/Users/ettor/Desktop/ugesco_file_temp_simplified.csv", encoding="utf-8")

In [8]:
# Merge with NER_classes (spatial_keys matched with wikidata)
ner_classes = pd.read_csv("data/ner_classes.csv")

data = pd.merge(data, ner_classes, how='left', left_on = ['spatial_key'], right_on = ['ner_class'])

In [9]:
# rename the new columns merged
data.rename(columns={'wiki_qid': 'ner_class_qid', 'wiki_class':'ner_class_name'}, inplace=True)

In [10]:
# remove articles in spatial_value
data['spatial_value'] = data['spatial_value'].str.replace(pat=r"^\s?(le|la|l\s+'|l'|les)\s+", repl="", n=1, case=False)

In [11]:
data.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name,ner_class,ner_class_qid,ner_class_name
0,00009500,Brussel,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Brussel. L'église St.e. Marie. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,214.0,bruxelles (belgique)--photographies,Q31,Belgium,Q239,Brussels,LOC_PLACE,Q56061,administrative territorial entity
1,00009500,église St . e. Marie,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Brussel. L'église St.e. Marie. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,214.0,bruxelles (belgique)--photographies,Q31,Belgium,Q239,Brussels,LOC_BUILDING,Q41176,building
2,00009520,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. Grand' Place.,Vues de Bruxelles : 1942-1944. - 124 photos,214.0,bruxelles (belgique)--photographies,Q31,Belgium,Q239,Brussels,LOC_PLACE,Q56061,administrative territorial entity
3,00009520,Grand ' Place,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. Grand' Place.,Vues de Bruxelles : 1942-1944. - 124 photos,214.0,bruxelles (belgique)--photographies,Q31,Belgium,Q239,Brussels,LOC_STREET,Q83620,thoroughfare
4,00009537,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. La Château de Tervuren. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,214.0,bruxelles (belgique)--photographies,Q31,Belgium,Q239,Brussels,LOC_PLACE,Q56061,administrative territorial entity


In [12]:
data.shape

(1166, 15)

In [13]:
data.to_csv(r"C:\Users\ettor\Desktop\data_ugesco.csv", header=True, index=False, encoding="utf-8")

In [ ]:
#test de stanford ner (lent)
#test = data.LEGEND[0:100].apply(get_ner)

In [ ]:
#data.LEGEND[0:100]

In [ ]:
#test_stanford = pd.concat([data.LEGEND[0:100], test], axis=1).reset_index()

In [ ]:
#test_stanford.to_csv("C:/Users/ettor/Desktop/test_stanford.csv")

#apply the algorithm
for index, rows in data.iterrows():
    dic = {}
    dic['index'] = index
    dic['beeldid'] = rows.beeldid
    if ("," not in str(rows.loc_wikiname) 
        and str(rows.loc_wikiname) != "nan"
        and isQidACity(rows.loc_qid)): 
        dic['city_thes_name'] = rows.loc_wikiname
        dic['city_qid_name'] = rows.loc_qid
        #dic['country_name'] = rows.country_name
        #dic['country_qid'] = rows.country_qid
    #elif (rows.spatial_key_0 == "LOC_PLACE" 
          #and isStringACity(rows.spatial_value_0)):
        #dic['city'] = rows.spatial_value_0
    print(dic)

          
        